In [7]:
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset
df = pd.read_csv(r"C:\Users\4t4\OneDrive\Desktop\time series for beds\holt's-W\WARD_daily.csv")

# Ensure that 'Date' is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Get a list of unique wards
wards = df['WARD'].unique()

# Initialize a DataFrame to store evaluation results for all wards
all_evaluation_results = pd.DataFrame()

# Wards to exclude
excluded_wards = ["KHCC-ADULT DAY CASE NZR", "KHCC-EMERGENCY KSB ROOM", "KHCC-BMT DAY CASE"]

for ward in wards:
    # Skip the excluded wards
    if ward in excluded_wards:
        print(f"Skipping ward: {ward}")
        continue

    print(f"\nProcessing ward: {ward}")

    # Filter the DataFrame to include only the data for the current ward
    df_ward = df[df['WARD'] == ward][['Date', 'TotalBedsUsed']].copy()

    # Check if there is enough data
    if len(df_ward) < 187:  # Need at least 7 points for forecasting and 180 points for testing
        print(f"Not enough data for ward {ward}. Skipping.")
        continue

    # Sort the DataFrame by 'Date'
    df_ward.sort_values('Date', inplace=True)

    # Set 'Date' as the index
    df_ward.set_index('Date', inplace=True)

    # Extract the 'TotalBedsUsed' series
    total_beds = df_ward['TotalBedsUsed']

    # Leave the last 180 days for testing
    train_data = total_beds.iloc[:-180]
    test_data = total_beds.iloc[-180:]

    # Initialize an empty DataFrame to store the forecasts and actuals for 180 days
    forecast_results = pd.DataFrame(index=test_data.index, columns=['Actual', 'Weekly', 'Daily', 'Yearly', 'All'])

    # Set actual values for comparison
    forecast_results['Actual'] = test_data

    # Step 2: Define Prophet models
    models = {
        'Weekly': Prophet(weekly_seasonality=True, yearly_seasonality=False, daily_seasonality=False),
        'Daily': Prophet(weekly_seasonality=False, yearly_seasonality=False, daily_seasonality=True),
        'Yearly': Prophet(weekly_seasonality=False, yearly_seasonality=True, daily_seasonality=False),
        'All': Prophet(weekly_seasonality=True, yearly_seasonality=True, daily_seasonality=True)
    }

    # Step 3: Perform forecasting over the 180 days using actual values
    for i in range(0, 180, 7):
        # Train data including up to the current window + actual values
        temp_train_data = total_beds.iloc[:-(180 - i)]  # Actual values used for training
        
        # Prepare data for Prophet
        train_df = pd.DataFrame({'ds': temp_train_data.index, 'y': temp_train_data.values})
        
        # Step 4: Refit the models on the updated train data and forecast
        for model_name, model in models.items():
            try:
                model.fit(train_df)
                future = model.make_future_dataframe(periods=7)
                forecast = model.predict(future)
                forecast_results[model_name].iloc[i:i+7] = forecast['yhat'].tail(7).values
            except Exception as e:
                print(f"Error fitting or forecasting with {model_name} model for ward {ward}: {e}")

    # Step 5: Plot the 180-day forecast vs actual values
    fig = go.Figure()

    # Add the actual data
    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Actual'], mode='lines+markers',
        name='Actual Data', line=dict(color='black')
    ))

    # Add the forecasted data
    colors = ['red', 'green', 'blue', 'orange']
    for i, model_name in enumerate(models.keys()):
        fig.add_trace(go.Scatter(
            x=forecast_results.index, y=forecast_results[model_name], mode='lines+markers',
            name=f'{model_name} Forecast', line=dict(color=colors[i], dash='dash')
        ))

    # Update layout
    fig.update_layout(
        title=f"Actual vs Forecast (180 Days) for Ward: {ward}",
        xaxis_title='Date',
        yaxis_title='Total Beds Used',
        showlegend=True
    )

    # Show the figure
    fig.show()

    # Step 6: Calculate MSE and MAE for the 180-day forecast
    try:
        # Drop rows with NaN values in any forecast column before calculating MSE and MAE
        valid_forecasts = forecast_results.dropna()

        # Now calculate the MSE and MAE for valid forecasts only
        mse_results = pd.DataFrame({
            'Model': list(models.keys()),
            'MSE': [mean_squared_error(valid_forecasts['Actual'], valid_forecasts[model]) for model in models.keys()],
            'MAE': [mean_absolute_error(valid_forecasts['Actual'], valid_forecasts[model]) for model in models.keys()]
        })

        # Append MSE and MAE results to the main evaluation DataFrame
        mse_results['Ward'] = ward
        all_evaluation_results = pd.concat([all_evaluation_results, mse_results], ignore_index=True)

    except Exception as e:
        print(f"Error calculating MSE and MAE for ward {ward}: {e}")

# Save the evaluation results to a CSV file
all_evaluation_results.to_csv('evaluation_results_all_wards_prophet.csv', index=False)
print("\nEvaluation results saved to 'evaluation_results_all_wards_prophet.csv'")

2024-10-14 00:17:28,514 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2024-10-14 00:17:28,570 - DEBUG - TBB already found in load path
2024-10-14 00:17:28,570 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2024-10-14 00:17:28,626 - DEBUG - TBB already found in load path
2024-10-14 00:17:28,626 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2024-10-14 00:17:28,677 - DEBUG - TBB already found in load path
2024-10-14 00:17:28,682 - DEBUG - cmd: where.exe tbb.dll
cwd: None



Processing ward: KHCC-10H-BMT


2024-10-14 00:17:28,733 - DEBUG - TBB already found in load path
2024-10-14 00:17:28,814 - DEBUG - input tempfile: C:\Users\4t4\AppData\Local\Temp\tmps9ugzi1w\2dkeago5.json
2024-10-14 00:17:28,828 - DEBUG - input tempfile: C:\Users\4t4\AppData\Local\Temp\tmps9ugzi1w\umpq9f6z.json
2024-10-14 00:17:28,828 - DEBUG - idx 0
2024-10-14 00:17:28,834 - DEBUG - running CmdStan, num_threads: None
2024-10-14 00:17:28,835 - DEBUG - CmdStan args: ['C:\\Users\\4t4\\anaconda3\\envs\\lstm_myenv\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=75473', 'data', 'file=C:\\Users\\4t4\\AppData\\Local\\Temp\\tmps9ugzi1w\\2dkeago5.json', 'init=C:\\Users\\4t4\\AppData\\Local\\Temp\\tmps9ugzi1w\\umpq9f6z.json', 'output', 'file=C:\\Users\\4t4\\AppData\\Local\\Temp\\tmps9ugzi1w\\prophet_modelq6lr3rxf\\prophet_model-20241014001728.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:17:28 - cmdstanpy - INFO - Chain [1] start processing
2024-10-14 00:17:28,836 - INFO - Chain [1]

Error fitting or forecasting with Weekly model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Daily model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Yearly model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with All model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Weekly model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Daily model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Yearly model for ward KHCC-10H-BMT: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with All model for ward KHCC-10H-BMT: Prophet

C:\Users\4t4\AppData\Local\Temp\ipykernel_6884\273995856.py:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

Skipping ward: KHCC-BMT DAY CASE
Skipping ward: KHCC-ADULT DAY CASE NZR

Processing ward: KHCC-5C-KOLAGHASSI WING


2024-10-14 00:17:56,005 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2024-10-14 00:17:56,058 - DEBUG - TBB already found in load path
2024-10-14 00:17:56,072 - DEBUG - input tempfile: C:\Users\4t4\AppData\Local\Temp\tmps9ugzi1w\7ro_swkc.json
2024-10-14 00:17:56,092 - DEBUG - input tempfile: C:\Users\4t4\AppData\Local\Temp\tmps9ugzi1w\xxv07as5.json
2024-10-14 00:17:56,092 - DEBUG - idx 0
2024-10-14 00:17:56,097 - DEBUG - running CmdStan, num_threads: None
2024-10-14 00:17:56,098 - DEBUG - CmdStan args: ['C:\\Users\\4t4\\anaconda3\\envs\\lstm_myenv\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=94671', 'data', 'file=C:\\Users\\4t4\\AppData\\Local\\Temp\\tmps9ugzi1w\\7ro_swkc.json', 'init=C:\\Users\\4t4\\AppData\\Local\\Temp\\tmps9ugzi1w\\xxv07as5.json', 'output', 'file=C:\\Users\\4t4\\AppData\\Local\\Temp\\tmps9ugzi1w\\prophet_modelgtsfz1qp\\prophet_model-20241014001756.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:17:56 - cmdstanpy - INFO - Ch

Error fitting or forecasting with Weekly model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Daily model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Yearly model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with All model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Weekly model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Daily model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new object.
Error fitting or forecasting with Yearly model for ward KHCC-5C-KOLAGHASSI WING: Prophet object can only be fit once. Instantiate a new objec

C:\Users\4t4\AppData\Local\Temp\ipykernel_6884\273995856.py:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




Evaluation results saved to 'evaluation_results_all_wards_prophet.csv'
